# Time for Crab

Age prediction of mud crabs based on physical features. [Dataset from Kaggle](https://www.kaggle.com/datasets/sidhus/crab-age-prediction).

## Dataset Columns

| Column Name    | Description                                                                                         |
|----------------|-----------------------------------------------------------------------------------------------------|
| Sex            | Gender of the Crab - Male, Female and Indeterminate.                                                |
| Length         | Length of the Crab (in Feet; 1 foot = 30.48 cms)                                                    |
| Diameter       | Diameter of the Crab (in Feet; 1 foot = 30.48 cms)                                                  |
| Height         | Height of the Crab (in Feet; 1 foot = 30.48 cms)                                                    |
| Weight         | Weight of the Crab (in ounces; 1 Pound = 16 ounces)                                                 |
| Shucked Weight | Weight without the shell (in ounces; 1 Pound = 16 ounces)                                           |
| Viscera Weight | is weight that wraps around your abdominal organs deep inside body (in ounces; 1 Pound = 16 ounces) |
| Shell Weight   | Weight of the Shell (in ounces; 1 Pound = 16 ounces)                                                |
| Age            | Age of the Crab (in months)                                                                         |


## Dataset Column Details

Sex

Gender of the Crab - Male, Female and Indeterminate.
M37%
I32%
Other (1225)31%
Valid
3893100%
Mismatched
00%
Missing
00%
Unique
3
Most Common
M37%

--- 

Length

Length of the Crab (in Feet; 1 foot = 30.48 cms)
Label	Count
0.19 - 0.37	7
0.37 - 0.56	56
0.56 - 0.74	138
0.74 - 0.93	286
0.93 - 1.11	412
1.11 - 1.30	728
1.30 - 1.48	980
1.48 - 1.67	956
1.67 - 1.85	301
1.85 - 2.04	29
0.19
2.04
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
1.31
Std. Deviation
0.3
Quantiles
0.19Min
1.1325%
1.3650%
1.5475%
2.04Max

---

Diameter

Diameter of the Crab (in Feet; 1 foot = 30.48 cms)
Label	Count
0.14 - 0.29	12
0.29 - 0.43	62
0.43 - 0.58	167
0.58 - 0.73	321
0.73 - 0.88	465
0.88 - 1.03	762
1.03 - 1.18	947
1.18 - 1.33	878
1.33 - 1.48	257
1.48 - 1.62	22
0.14
1.63
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
1.02
Std. Deviation
0.25
Quantiles
0.14Min
0.8825%
1.0650%
1.275%
1.63Max

--- 

Height

Height of the Crab (in Feet; 1 foot = 30.48 cms)
Label	Count
0.00 - 0.28	938
0.28 - 0.57	2,934
0.57 - 0.85	19
1.13 - 1.41	1
2.54 - 2.83	1
0
2.83
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
0.35
Std. Deviation
0.1
Quantiles
0Min
0.2925%
0.3650%
0.4175%
2.83Max

---

Weight

Weight of the Crab (in ounces; 1 Pound = 16 ounces)
Label	Count
0.06 - 8.06	593
8.06 - 16.07	716
16.07 - 24.07	767
24.07 - 32.07	773
32.07 - 40.08	585
40.08 - 48.08	264
48.08 - 56.09	123
56.09 - 64.09	50
64.09 - 72.10	16
72.10 - 80.10	6
0.06
80.1
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
23.6
Std. Deviation
13.9
Quantiles
0.06Min
12.725%
22.850%
32.875%
80.1Max

---

Shucked Weight

Weight without the shell (in ounces; 1 Pound = 16 ounces)
Label	Count
0.03 - 4.24	730
4.24 - 8.46	975
8.46 - 12.68	891
12.68 - 16.89	736
16.89 - 21.11	372
21.11 - 25.32	117
25.32 - 29.54	40
29.54 - 33.75	23
33.75 - 37.97	6
37.97 - 42.18	3
0.03
42.2
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
10.2
Std. Deviation
6.27
Quantiles
0.03Min
5.3425%
9.5450%
14.375%
42.2Max

---

Viscera Weight

is weight that wraps around your abdominal organs deep inside body (in ounces; 1 Pound = 16 ounces)
Label	Count
0.01 - 2.17	772
2.17 - 4.32	920
4.32 - 6.47	958
6.47 - 8.63	716
8.63 - 10.78	339
10.78 - 12.93	133
12.93 - 15.09	46
15.09 - 17.24	7
17.24 - 19.39	1
19.39 - 21.55	1
0.01
21.5
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
5.14
Std. Deviation
3.1
Quantiles
0.01Min
2.6625%
4.8650%
7.275%
21.5Max

--- 

Shell Weight

Weight of the Shell (in ounces; 1 Pound = 16 ounces)
Label	Count
0.04 - 2.89	720
2.89 - 5.73	943
5.73 - 8.58	1,011
8.58 - 11.42	754
11.42 - 14.27	321
14.27 - 17.11	99
17.11 - 19.96	31
19.96 - 22.80	9
22.80 - 25.65	4
25.65 - 28.49	1
0.04
28.5
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
6.8
Std. Deviation
3.94
Quantiles
0.04Min
3.7125%
6.6650%
9.3675%
28.5Max

---

Age

Age of the Crab (in months)
Label	Count
1.00 - 3.80	15
3.80 - 6.60	401
6.60 - 9.40	1,517
9.40 - 12.20	1,305
12.20 - 15.00	317
15.00 - 17.80	212
17.80 - 20.60	92
20.60 - 23.40	27
23.40 - 26.20	4
26.20 - 29.00	3
1
29
Valid
3893100%
Mismatched
00%
Missing
00%
Mean
9.95
Std. Deviation
3.22
Quantiles
1Min
825%
1050%
1175%
29Max

